## 🧠💡 Intelligent Systems  for Smart Health 👨‍⚕👩‍⚕️🔬🌡️

# Risk Models Using Tree-based Models

In a first step, we used linear models (Linear regression, logistic regression). Here we will explore another basic type of **machine learning** models: **tree-based models**!

We will work with actual medical data in this notebook, namely the NHANES I epidemiology dataset (for a detailed description of this dataset you can check the [CDC Website](https://wwwn.cdc.gov/nchs/nhanes/nhefs/default.aspx/)).

The key aim here is to predict the 10-year risk of death of individuals from the study. 



## Table of Contents

- [1. Import Packages](#1)
- [2. The Dataset](#2)
    - [Exercise 1](#ex1)
- [3. Handle missing data](#3)
- [4. Decision Trees](#4)
    - [Exercise 2 - hyperparmaters](#ex2)
- [5. Random Forests](#5)
    - [Exercise 3 - hyperparameter optimization](#ex-3)
- [6. Systematic hyperparameter search](#6)


<a name='1'></a>
## 1. Import Packages

We'll first import all the common packages that we need for this assignment. 

- `sklearn` is one of the most popular machine learning libraries.
- `itertools` allows us to conveniently manipulate iterable objects such as lists.
- `pydotplus` is used together with `IPython.display.Image` to visualize graph structures such as decision trees.
- `numpy` is a fundamental package for scientific computing in Python.
- `pandas` is what we'll use to manipulate our data.
- `seaborn` is a plotting library which has some convenient functions for visualizing missing data.
- `matplotlib` is a plotting library.

In [ ]:
# maybe some missing libraries
#!pip install pydotplus
#!pip install lifelines

In [ ]:
import os
import sklearn
import itertools
import pydotplus
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.display import Image 

from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

<a name='2'></a>
## 2. The Dataset
### Load and explore the data!

In virtually all cases, we would first want to get an intuition on the data itself. Things like: What is in the data? How much data is there? Are there things missing? What might cause problems? Do we understand the type of data/features?

With **pandas**, we usually can explore some key properties very rapidly, for instance with commands like

- `data.head()`
- `data.describe()`
- `data.info()`

### Some weird conventions:
For some reason it became standard in the machine learning world to name the actual data `X` and the labels `y`. Even though I consider this a rather poor choice both from a math and from a code best practice point of view, we will stick to this in this notebook.

In [ ]:
path_data = "../data/"  # add your own path

X = pd.read_csv(os.path.join(path_data, "NHANESI_subset_X.csv"))
y = pd.read_csv(os.path.join(path_data, "NHANESI_subset_y.csv"))

X.head()

In [ ]:
y.head()

<a name='ex1'></a>
## Exercise 1:

Answer the following questions:
- How many features do we have?
- How many datapoints do we have?
- What is our label?
- Do we have missing data?

### Remove the "unnamed" columns

<a name='3'></a>
## 3. Handle missing data

Whenever we face missing data (incomplete entries, NaNs etc.), we have to decide how to deal with this. In general there are the following options:

1. Ignore (usually a very poor choice... **unless**: we decide to deal with it later!)
2. Remove incomplete entries --> Rows (fast and clean, but we loose datapoints)
3. Remove incomplete entries --> Columns/Features (fast and clean, but we loose potentially important features)
4. Replace missing entries --> replace by what? Average values, zeros, best guesses...? Can be an option, but needs a good understanding of the data and the process!

#### Filling and dropping with pandas
We can remove NaNs with pandas using `data.dropna()`. Or we can fill NaN entries using `data.fillna()`. Important here is to specify along which `axis` this operation should be done (rows or columns?).

### Looks good... or maybe not?

- Now we have no incomplete entries in the data. But something went **VERY** wrong here. What is it?

### Label is not fully understandable without further information

For now let's simply say that all values > 0 mean: person died after x years.
Negative values mean that those people haven't died as long as they participated (which was -years).
In the following we aim to predict if people died after >= 10 years.

In [ ]:
threshold = 10  # we focus on >10 year risk

data["death"] = np.ones(len(data))

# remove all people which haven't died within the study
data.loc[data["time"] < 0, "death"] = 0

# remove data which we can (and should) not use for predicting >10 year risk
mask = (data["time"] > 0) | (data["time"] <= -threshold)
data = data[mask]

# Create data/label split --> X, y
X = data.drop(["death", "time"], axis='columns')
y = (data.time < threshold) & (data.time > 0)  # died within 0 and threshold years

### Inspect again data (X) and labels (y)
- how much data have we left?
- how are the labels distributed?
- how is the "Age" and "Race" distributed?

# Train/Test split
### --> For now: ignore biases

Ways to better take care of those include:
- random sampling with weights
- model training with weights
- over- or under-sampling

### Train / Validation / Test

We already discussed spliting the data into a **training set** and a **test set**. Often, however, we want to have 2 different test sets. One which is the "real" test set and remains untouched and is reserved for the model evaluation. And one set which we use to optimize our model parameters. The second one is typically called **validation set**.

In [ ]:
# first split --> get test set
X_dev, X_test, y_dev, y_test = ...

In [ ]:
# second split --> train / validation
X_train, X_val, y_train, y_val = ...

Use the next cell to look at an individual case get familiar with the features.

In [ ]:
i = 3
print(X_train.iloc[i,:])
print("\nDied within 10 years? {}".format(y_train.loc[y_train.index[i]]))

<a name='4'></a>
## 4. Training a decision tree model

Scikit-learn provides numerous differnt machine learning models and tools. This is very convenient to use, in particular because many elements of scikit-learn can be used in (more or less) the same way. 

For more details, please look at the [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

The model we will use first in this notebook is a **decision tree classifier** (`DecisionTreeClassifier()`). The training of a model is again simply done by running `model.fit()`.

- Please train such a decision tree using the training set.

In [ ]:
tree = DecisionTreeClassifier(random_state=10)
# train the model

### Two types of predictions
- final predicition / classification
- "probability" (not a real probability, but still...)

#### 1. Use the model to make a prediciton

Often we can also access the output one step before the final "decision". For some models this will give us float values ("probabilites"... kind of). Here, because we use one single decision tree we will only get 1 or 0 values.

#### 2. Do the same but using `.predict_proba)
What do you get from this method?

#### Compute a confusion matrix using the training set

In [ ]:
from sklearn.metrics import confusion_matrix

# ...

#### Let's check again on the validation set (X_val)

### Alternative evaluation metric: concordance

We can also use concordance, or the C-Index for evaluation.

The C-Index evaluates the ability of a model to differentiate between different classes, by quantifying how often, when considering all pairs of patients (A, B), the model says that patient A has a higher risk score than patient B when, in the observed data, patient A actually died and patient B actually lived. In our case, our model is a binary classifier, where each risk score is either 1 (the model predicts that the patient will die) or 0 (the patient will live).

More formally, defining _permissible pairs_ of patients as pairs where the outcomes are different, _concordant pairs_ as permissible pairs where the patient that died had a higher risk score (i.e. our model predicted 1 for the patient that died and 0 for the one that lived), and _ties_ as permissible pairs where the risk scores were equal (i.e. our model predicted 1 for both patients or 0 for both patients), the C-Index is equal to:

$$\text{C-Index} = \frac{\#\text{concordant pairs} + 0.5\times \#\text{ties}}{\#\text{permissible pairs}}$$



In [ ]:
import lifelines

def cindex(y_true, scores):
    return lifelines.utils.concordance_index(y_true, scores)

In [ ]:
# How good are the prediction according to the c-index?
y_train_preds = tree.predict_proba(X_train)[:, 1]
print(f"Train C-Index: {cindex(y_train.values, y_train_preds)}")

y_val_preds = tree.predict_proba(X_val)[:, 1]
print(f"Val C-Index: {cindex(y_val.values, y_val_preds)}")

<a name='ex-2'></a>
### Exercise 2 - decision tree hyperparameters

Try and find a set of hyperparameters that improves the generalization to the validation set and recompute the C-index. If you do it right, you should get C-index above 0.6 for the validation set. 

You can refer to the documentation for the sklearn [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [ ]:
tree = DecisionTreeClassifier(max_depth=...)
# train your model

In [ ]:
# Inspect the results
y_train_preds = tree.predict_proba(X_train)[:, 1]
print(f"Train C-Index: {cindex(y_train.values, y_train_preds)}")

y_val_preds = tree.predict_proba(X_val)[:, 1]
print(f"Val C-Index: {cindex(y_val.values, y_val_preds)}")

### Inspect the trained decision tree
One of the biggest advantages of a **decision tree** model is that we can intuitively understand and inspect how it comes to its predictions.

We can, for instance, output the learned tree to see how decisions are made.

In [ ]:
from sklearn.tree import plot_tree

fig = plt.figure(figsize=(15, 7), dpi=300)
plot_tree(tree, 
          feature_names=X_train.columns,  
          class_names=['neg', 'pos'],
          filled=True)

#plt.savefig("decision_tree.pdf")
plt.show()